In [ ]:
! pip install numba

     |████████████████████████████████| 3.3 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 47.0 MB/s eta 0:00:01


In [ ]:
! pip install fastcore rasterio timm wandb --upgrade
! pip install kornia --no-deps --upgrade
! pip install fastai.zip
! pip install fastxtend.zip

     |████████████████████████████████| 56 kB 2.1 MB/s eta 0:00:011
     |████████████████████████████████| 19.3 MB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 431 kB 82.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.7 MB/s s eta 0:00:01
     |████████████████████████████████| 1.1 MB 79.7 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 86.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 180 kB 72.2 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.2 MB/s s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=3864b80c5be370fd41114bb480bc4332d983f7c6b159b215acf6de8433965808
  Stored in directory: /root/.cache/pip/wheels/29/93/c6/762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
  Created wheel for subprocess32: filename=subprocess3

In [ ]:
%env WANDB_SILENT=true

env: WANDB_SILENT=true


In [ ]:
import wandb

In [ ]:
wandb.login()

True

In [ ]:
from fastai.test_utils import show_install
show_install(True)



```text
=== Software === 
python        : 3.7.11
fastai        : 2.5.4
fastcore      : 1.3.27
fastprogress  : 0.2.7
torch         : 1.10.0
nvidia driver : 495.46
torch cuda    : 11.3 / is available
torch cudnn   : 8200 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : NVIDIA GeForce RTX 3090

=== Environment === 
platform      : Linux-5.11.0-46-generic-x86_64-with-debian-buster-sid
distro        : #51~20.04.1-Ubuntu SMP Fri Jan 7 06:51:40 UTC 2022
conda env     : Unknown
python        : /opt/conda/bin/python
sys.path      : /workspace
/opt/conda/lib/python37.zip
/opt/conda/lib/python3.7
/opt/conda/lib/python3.7/lib-dynload

/opt/conda/lib/python3.7/site-packages
/opt/conda/lib/python3.7/site-packages/IPython/extensions
/root/.ipython

Mon Jan 17 23:24:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|--------------------------

# Imports

In [ ]:
from fastai.vision.all import *
import rasterio
import gc
from numba import jit
import torch.nn.functional as F
from fastxtend.augment import tensor_item_tfm
from fastxtend.schedulers import fit_flat_varied
from fastxtend.utils.lessrandom import *
from fastai.callback.wandb import WandbCallback

In [ ]:
FULL = True

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

# New Datablocks

In [ ]:
PATH = '' # Colab
#PATH = 'data' # Test env

LABELS       = f'{PATH}train_labels/'
FEATURES     = f'{PATH}train_features/'
if FULL:
    NPY_LABELS   = f'{PATH}npy_labels/'
    NPY_FEATURES = f'{PATH}npy_features/'
else:
    NPY_LABELS   = f'{PATH}sub_labels/'
    NPY_FEATURES = f'{PATH}sub_features/'


# from https://github.com/makepath/xarray-spatial/blob/master/xrspatial/multispectral.py
@jit(nopython=True, nogil=True)
def _normalize_data_cpu(data, min_val, max_val, pixel_max, c, th):
    out = np.zeros_like(data)
    out[:] = np.nan

    range_val = max_val - min_val
    rows, cols = data.shape

    # check range_val to avoid dividing by zero
    if range_val != 0:
        for y in range(rows):
            for x in range(cols):
                val = data[y, x]
                norm = (val - min_val) / range_val
                # sigmoid contrast enhancement
                norm = 1 / (1 + np.exp(c * (th - norm)))
                out[y, x] = norm * pixel_max
    return out

def _normalize_data(data, pixel_max, c, th):
    min_val = np.nanmin(data)
    max_val = np.nanmax(data)
    return _normalize_data_cpu(data, min_val, max_val, pixel_max, c, th)

# end xarray-spatial code


def _read_tif(fn):
    with rasterio.open(f'{fn}/B04.tif') as img:
        r_img = img.read(1).astype('float32')
    with rasterio.open(f'{fn}/B03.tif') as img:
        g_img = img.read(1).astype('float32')
    with rasterio.open(f'{fn}/B02.tif') as img:
        b_img = img.read(1).astype('float32')
    with rasterio.open(f'{fn}/B08.tif') as img:
        i_img = img.read(1).astype('float32')
    return np.stack([r_img, g_img, b_img, i_img], axis=0)


class TensorImageGeo(TensorImage):
    @classmethod
    def create_tif(cls, fn, **kwargs)->None:
        "Open an `Image` from path `fn`"
        return  cls(torch.from_numpy(_read_tif(fn)))

    @classmethod
    def create_npy(cls, fn, **kwargs)->None:
        "Open an `Image` from path `fn`"
        return cls(torch.from_numpy(np.load(f'{NPY_FEATURES}{fn}.npy')).float())

    def show(self, ctx=None, **kwargs):
        "Show image using `merge(self._show_args, kwargs)`"
        return show_image(self._true_color(), ctx=ctx, **{**self._show_args, **kwargs})

    # adapted from https://github.com/makepath/xarray-spatial/blob/master/xrspatial/multispectral.py
    def _true_color(self, nodata=1, c=10.0, th=0.125, pixel_max = 255):
        img = self.detach().clone().cpu().numpy()
        img[0, :, :] = _normalize_data(img[0, :, :], pixel_max, c, th)
        img[1, :, :] = _normalize_data(img[1, :, :], pixel_max, c, th)
        img[2, :, :] = _normalize_data(img[2, :, :], pixel_max, c, th)
        return img[0:3, :, :].transpose(1,2,0).astype(np.uint8)


class TensorMaskGeo(TensorMask):
    _show_args = {'alpha':0.25, 'cmap':'cool', 'interpolation':'nearest'}
    @classmethod
    def create_tif(cls, fn)->None:
        "Open an `Image` from path `fn`"
        with rasterio.open(f'{LABELS}{fn}.tif') as img:
            lbl = img.read(1)
        return cls(torch.from_numpy(lbl))

    @classmethod
    def create_npy(cls, fn, **kwargs)->None:
        "Open an `Image` from path `fn`"
        return cls(torch.from_numpy(np.load(f'{NPY_LABELS}{fn}.npy')))


def _four_corners(img):
    imgh, imgw = img.shape[-2], img.shape[-1]
    croph, cropw = imgh//2, imgw//2
    tl = img[..., 0:croph, 0:cropw]
    tr = img[..., 0:croph, imgw-cropw:imgw]
    bl = img[..., imgh-croph:imgh, 0:cropw]
    br = img[..., imgh-croph:imgh, imgw-cropw:imgw]
    return torch.stack([tl, tr, bl, br], 0)


class TensorImageGeoCrop(TensorImageGeo):
    @classmethod
    def create_tif(cls, fn, **kwargs)->None:
        "Open an `Image` from path `fn`"
        return cls(_four_corners(torch.from_numpy(_read_tif(fn))))

    @classmethod
    def create_npy(cls, fn, **kwargs)->None:
        "Open an `Image` from path `fn`"
        return cls(_four_corners(torch.from_numpy(np.load(f'{NPY_FEATURES}{fn}.npy')).float()))


class TensorMaskGeoCrop(TensorMaskGeo):
    @classmethod
    def create_tif(cls, fn)->None:
        "Open an `Image` from path `fn`"
        with rasterio.open(f'{LABELS}{fn}.tif') as img:
            lbl = img.read(1)
        return cls(_four_corners(torch.from_numpy(lbl)))

    @classmethod
    def create_npy(cls, fn, **kwargs)->None:
        "Open an `Image` from path `fn`"
        return cls(_four_corners(torch.from_numpy(np.load(f'{NPY_LABELS}{fn}.npy'))))



class TensorMaskToClass(TensorMaskGeo):
    pass

class TensorCropMaskToClass(TensorMaskGeoCrop):
    pass


@ToTensor
def encodes(self, o:TensorImageGeo):
    return o

@ToTensor
def encodes(self, o:TensorMaskGeo):
    return o


class Normalize(DisplayedTransform):
    "Normalize/denorm batch of `TensorImage`"
    parameters,order = L('mean', 'std'),99
    def __init__(self, mean=None, std=None, axes=(0,2,3)): store_attr()

    @classmethod
    def from_stats(cls, mean, std, dim=1, ndim=4, cuda=True): return cls(*broadcast_vec(dim, ndim, mean, std, cuda=cuda))

    def setups(self, dl:DataLoader):
        if self.mean is None or self.std is None:
            x,*_ = dl.one_batch()
            self.mean,self.std = x.mean(self.axes, keepdim=True),x.std(self.axes, keepdim=True)+1e-7

    def encodes(self, x:TensorImageGeo): return (x-self.mean) / self.std

    def decodes(self, x:TensorImageGeo):
        f = to_cpu if x.device.type=='cpu' else noop
        return (x*f(self.std) + f(self.mean))


class MaskToMultiClass(DisplayedTransform):
    order=100
    def encodes(self, x:(TensorMaskToClass, TensorCropMaskToClass)):
        s = x.shape
        x = x.sum(axis=(1,2)) / (s[1]*s[2])
        y = torch.zeros([x.shape[0], 20], dtype=x.dtype, device=x.device)
        for i in range(len(x)):
            z = torch.ones(int(20 * x[i]), dtype=x.dtype, device=x.device)
            z = F.pad(z, [0, 20-z.shape[0]])
            y[i] = z
        return TensorMultiCategory(y)
    def decodes(self, x:TensorMultiCategory):
        f = to_cpu if x.device.type=='cpu' else noop
        return f(x.sum(dim=1))

@patch
def show(self:TensorMultiCategory, ctx=None, **kwargs):
    "Show self"
    return show_title(str(self.item()), ctx=ctx, **merge({'label': 'text'}, kwargs))


class ReshapeCrop(DisplayedTransform):
    order = 9 # Runs right before IntToFloatTensor on the GPU
    def encodes(self, x:TensorImageGeoCrop):
        s = x.shape
        return x.view(s[0]*s[1],s[2],s[3],s[4])

    def encodes(self, x:TensorMaskGeoCrop):
        s = x.shape
        return x.view(s[0]*s[1],s[2],s[3])

    def decodes(self, x:TensorImageGeoCrop):
        xs = []
        for i in range(0, find_bs(x), 4):
            xs.append(make_grid(x[i:i+4], nrow=2, padding=0))
        return torch.stack(xs, dim=0)


def GeoImageBlock(cls=TensorImageGeo, tif=True):
    if tif: return TransformBlock(type_tfms=cls.create_tif)
    else:   return TransformBlock(type_tfms=cls.create_npy)

def GeoMaskBlock(codes=None, tif=True):
    if tif: return TransformBlock(type_tfms=TensorMaskGeo.create_tif)
    else:   return TransformBlock(type_tfms=TensorMaskGeo.create_npy)

def GeoImageCropBlock(cls=TensorImageGeoCrop, tif=True):
    if tif: return TransformBlock(type_tfms=cls.create_tif, batch_tfms=ReshapeCrop)
    else:   return TransformBlock(type_tfms=cls.create_npy, batch_tfms=ReshapeCrop)

def GeoMaskCropBlock(cls=TensorMaskGeoCrop, tif=True):
    if tif: return TransformBlock(type_tfms=cls.create_tif, batch_tfms=ReshapeCrop)
    else:   return TransformBlock(type_tfms=cls.create_npy, batch_tfms=ReshapeCrop)

def GeoMultiClassBlock(cls=TensorMaskToClass, codes=None, tif=True):
    if tif: return TransformBlock(type_tfms=cls.create_tif, batch_tfms=MaskToMultiClass)
    else:   return TransformBlock(type_tfms=cls.create_npy, batch_tfms=MaskToMultiClass)

def GeoCropMultiClassBlock(cls=TensorCropMaskToClass, codes=None, tif=True):
    if tif: return TransformBlock(type_tfms=cls.create_tif, batch_tfms=[ReshapeCrop, MaskToMultiClass])
    else:   return TransformBlock(type_tfms=cls.create_npy, batch_tfms=[ReshapeCrop, MaskToMultiClass])


In [ ]:
from fastai.callback.wandb import _make_plt

@typedispatch
def wandb_process(x:TensorImageGeo, y, samples, outs):
    "Process `sample` and `out` depending on the type of `x/y`"
    res_input, res_pred, res_label = [],[],[]
    for s,o in zip(samples, outs):
        img = TensorImage(s[0]._true_color())
        res_input.append(wandb.Image(img, caption='Input data'))
        for t, capt, res in ((o[0], "Prediction", res_pred), (s[1], "Ground Truth", res_label)):
            fig, ax = _make_plt(img)
            # Superimpose label or prediction to input image
            ax = img.show(ctx=ax)
            ax = t.show(ctx=ax)
            res.append(wandb.Image(fig, caption=capt))
            plt.close(fig)
    return {"Inputs":res_input, "Predictions":res_pred, "Ground Truth":res_label}

# Callbacks

In [ ]:
from torchvision.utils import make_grid

class CombineGeoCallback(Callback):
    run_train,order = False,MixedPrecision.order+1

    def after_pred(self):
        preds = []
        for i in range(0, find_bs(self.learn.pred), 4):
            preds.append(make_grid(self.learn.pred[i:i+4], nrow=2, padding=0))
        self.learn.pred = TensorImageGeo(torch.stack(preds, dim=0))

In [ ]:
# from https://nicjac.dev/posts/identify-best-model/ soon to be added to fast.ai
class SaveModelCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model's best during training and loads it at the end."
    order = TrackerCallback.order+1
    def __init__(self, monitor='valid_loss', comp=None, min_delta=0., fname='model', every_epoch=False, at_end=False,
                 with_opt=False, reset_on_fit=True):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta, reset_on_fit=reset_on_fit)
        assert not (every_epoch and at_end), "every_epoch and at_end cannot both be set to True"
        # keep track of file path for loggers
        self.last_saved_path = None
        self.last_saved_metadata = None
        
        store_attr('fname,every_epoch,at_end,with_opt')

    def _save(self, name, metadata): 
        self.last_saved_path = self.learn.save(name, with_opt=self.with_opt)
        self.last_saved_metadata = metadata

    def after_epoch(self):
        "Compare the value monitored to its best score and save if best."
        if self.every_epoch:
            if (self.epoch%self.every_epoch) == 0: self._save(f'{self.fname}_{self.epoch}')
        else: #every improvement
            super().after_epoch()
            if self.new_best:
                print(f'Better model found at epoch {self.epoch} with {self.monitor} value: {self.best}.')
                self._save(f'{self.fname}', {n:s for n,s in zip(self.recorder.metric_names, self.recorder.log)})

    def after_fit(self, **kwargs):
        "Load the best model."
        if self.at_end: self._save(f'{self.fname}')
        elif not self.every_epoch: self.learn.load(f'{self.fname}', with_opt=self.with_opt)

In [ ]:
# from https://nicjac.dev/posts/identify-best-model/ soon to be added to fast.ai
from fastai.callback.wandb import log_model

@patch
def after_fit(self:WandbCallback):
    if self.log_model:
        if self.save_model.last_saved_path is None:
            print('WandbCallback could not retrieve a model to upload')
        else:
            log_model(self.save_model.last_saved_path, metadata=self.save_model.last_saved_metadata)
            
            for metadata_key in self.save_model.last_saved_metadata:
                wandb.run.summary[f'best_{metadata_key}'] = self.save_model.last_saved_metadata[metadata_key]

# Models

## Pooling

In [ ]:
from kornia.filters import MaxBlurPool2D

stem_pool  = partial(MaxBlurPool2D, kernel_size=3, ceil_mode=True)
block_pool = partial(MaxBlurPool2D, kernel_size=3, ceil_mode=True)

## XResNet

In [ ]:
from torch.nn.parameter import Parameter

class sa_layer(nn.Module):
    """Constructs a Shuffle Attention module.
    Args:
        channel: number of input channels
    """
    def __init__(self, channel, groups=64):
        super(sa_layer, self).__init__()
        self.groups = groups
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.cweight = Parameter(torch.zeros(1, channel // (2 * groups), 1, 1))
        self.cbias = Parameter(torch.ones(1, channel // (2 * groups), 1, 1))
        self.sweight = Parameter(torch.zeros(1, channel // (2 * groups), 1, 1))
        self.sbias = Parameter(torch.ones(1, channel // (2 * groups), 1, 1))

        self.sigmoid = nn.Sigmoid()
        self.gn = nn.GroupNorm(channel // (2 * groups), channel // (2 * groups))

    @staticmethod
    def channel_shuffle(x, groups):
        b, c, h, w = x.shape

        x = x.reshape(b, groups, -1, h, w)
        x = x.permute(0, 2, 1, 3, 4)

        # flatten
        x = x.reshape(b, -1, h, w)

        return x

    def forward(self, x):
        b, c, h, w = x.shape

        x = x.reshape(b * self.groups, -1, h, w)
        x_0, x_1 = x.chunk(2, dim=1)

        # channel attention
        xn = self.avg_pool(x_0)
        xn = self.cweight * xn + self.cbias
        xn = x_0 * self.sigmoid(xn)

        # spatial attention
        xs = self.gn(x_1)
        xs = self.sweight * xs + self.sbias
        xs = x_1 * self.sigmoid(xs)

        # concatenate along channel axis
        out = torch.cat([xn, xs], dim=1)
        out = out.reshape(b, -1, h, w)

        out = self.channel_shuffle(out, 2)
        return out

In [ ]:
from fastai.torch_basics import *
from fastai.layers import ConvLayer, NormType, SimpleSelfAttention
from timm.models.layers.eca import EcaModule


def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv1d,nn.Conv2d,nn.Conv3d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)


class ResBlock(Module):
    "Resnet block from `ni` to `nh` with `stride`"
    @delegates(ConvLayer.__init__)
    def __init__(self, expansion, ni, nf, stride=1, groups=1, reduction=None, sha=False, eca=False, 
                 nh1=None, nh2=None, dw=False, g2=1, sa=False, sym=False, norm_type=NormType.Batch, 
                 act_cls=defaults.activation, ndim=2, ks=3, pool=AvgPool, pool_first=True, block_pool=None, **kwargs):
        norm2 = (NormType.BatchZero if norm_type==NormType.Batch else
                 NormType.InstanceZero if norm_type==NormType.Instance else norm_type)
        if nh2 is None: nh2 = nf
        if nh1 is None: nh1 = nh2
        nf,ni = nf*expansion,ni*expansion
        k0 = dict(norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs)
        k1 = dict(norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        convpath  = [ConvLayer(ni,  nh2, ks, stride=stride, groups=ni if dw else groups, **k0),
                     ConvLayer(nh2,  nf, ks, groups=g2, **k1)
        ] if expansion == 1 else [
                     ConvLayer(ni,  nh1, 1, **k0),
                     ConvLayer(nh1, nh2, ks, stride=stride, groups=nh1 if dw else groups, **k0),
                     ConvLayer(nh2,  nf, 1, groups=g2, **k1)]
        if reduction: convpath.append(SEModule(nf, reduction=reduction, act_cls=act_cls))
        if sha: convpath.append(sa_layer(nf))
        if eca: convpath.append(EcaModule(nf))
        if sa: convpath.append(SimpleSelfAttention(nf,ks=1,sym=sym))
        self.convpath = nn.Sequential(*convpath)
        idpath = []
        if ni!=nf: idpath.append(ConvLayer(ni, nf, 1, act_cls=None, ndim=ndim, **kwargs))
        if stride!=1:
            if block_pool==None:
                idpath.insert((1,0)[pool_first], pool(stride, ndim=ndim, ceil_mode=True))
            else:
                idpath.insert((1,0)[pool_first], block_pool())
        self.idpath = nn.Sequential(*idpath)
        self.act = defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls()

    def forward(self, x): return self.act(self.convpath(x) + self.idpath(x))


class XResNet(Module):
    @delegates(ResBlock)
    def __init__(self, block, expansion, layers, p=0.0, c_in=3, n_out=1000, stem_szs=(32,32,64),
                 widen=1.0, sa=False, act_cls=defaults.activation, norm_type=NormType.Batch, ndim=2,  
                 ks=3, stride=2, stem_pool=partial(MaxPool, ks=3, stride=2, padding=3//2, ndim=2), **kwargs):
        store_attr('block,expansion,act_cls,ndim,ks')
        if ks % 2 == 0: raise Exception('kernel size has to be odd!')
        stem_szs = [c_in, *stem_szs]
        stem = [ConvLayer(stem_szs[i], stem_szs[i+1], ks=ks, stride=stride if i==0 else 1,
                          act_cls=act_cls, ndim=ndim, norm_type=norm_type)
                for i in range(3)]

        self.stem=nn.Sequential(
            *stem, stem_pool())   

        block_szs   = [int(o*widen) for o in [64,128,256,512] +[256]*(len(layers)-4)]
        block_szs   = [64//expansion] + block_szs
        blocks = self._make_blocks(layers, block_szs, sa, stride, norm_type=norm_type, **kwargs)
        
        # quick fix to get working
        self.layer1 = blocks[0] 
        self.layer2 = blocks[1]
        self.layer3 = blocks[2]
        self.layer4 = blocks[3]

        self.head = nn.Sequential(AdaptiveAvgPool(sz=1, ndim=ndim), Flatten(), nn.Dropout(p),
                                  nn.Linear(block_szs[-1]*expansion, n_out))

        init_cnn(self)

    def _make_blocks(self, layers, block_szs, sa, stride, **kwargs):
        return [self._make_layer(ni=block_szs[i], nf=block_szs[i+1], blocks=l,
                                 stride=1 if i==0 else stride, sa=sa and i==len(layers)-4, **kwargs)
                for i,l in enumerate(layers)]

    def _make_layer(self, ni, nf, blocks, stride, sa, **kwargs):
        return nn.Sequential(
            *[self.block(self.expansion, ni if i==0 else nf, nf, stride=stride if i==0 else 1,
                      sa=sa and i==(blocks-1), act_cls=self.act_cls, ndim=self.ndim, ks=self.ks, **kwargs)
              for i in range(blocks)])
        
    def forward(self, x):
        x = self.stem(x)

        c1 = self.layer1(x)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)

        return self.head(c4)


class XResNetDeepLabHead(Module):
    @delegates(ResBlock)
    def __init__(self, block, expansion, layers, p=0.0, c_in=3, n_out=1000, stem_szs=(32,32,64),
                 widen=1.0, sa=False, act_cls=defaults.activation, norm_type=NormType.Batch, ndim=2,  
                 ks=3, stride=2, stem_pool=partial(MaxPool, ks=3, stride=2, padding=3//2, ndim=2), **kwargs):
        store_attr('block,expansion,act_cls,ndim,ks')
        if ks % 2 == 0: raise Exception('kernel size has to be odd!')
        stem_szs = [c_in, *stem_szs]
        stem = [ConvLayer(stem_szs[i], stem_szs[i+1], ks=ks, stride=stride if i==0 else 1,
                          act_cls=act_cls, ndim=ndim, norm_type=norm_type)
                for i in range(3)]

        self.stem=nn.Sequential(
            *stem, stem_pool())   

        block_szs   = [int(o*widen) for o in [64,128,256,512] +[256]*(len(layers)-4)]
        block_szs   = [64//expansion] + block_szs
        blocks = self._make_blocks(layers, block_szs, sa, stride, norm_type=norm_type, **kwargs)
        
        # quick fix to get working
        self.layer1 = blocks[0] 
        self.layer2 = blocks[1]
        self.layer3 = blocks[2]
        self.layer4 = blocks[3]

        # self.head = nn.Sequential(AdaptiveAvgPool(sz=1, ndim=ndim), Flatten(), nn.Dropout(p),
        #                           nn.Linear(block_szs[-1]*expansion, n_out))

        init_cnn(self)

    def _make_blocks(self, layers, block_szs, sa, stride, **kwargs):
        return [self._make_layer(ni=block_szs[i], nf=block_szs[i+1], blocks=l,
                                 stride=1 if i==0 else stride, sa=sa and i==len(layers)-4, **kwargs)
                for i,l in enumerate(layers)]

    def _make_layer(self, ni, nf, blocks, stride, sa, **kwargs):
        return nn.Sequential(
            *[self.block(self.expansion, ni if i==0 else nf, nf, stride=stride if i==0 else 1,
                      sa=sa and i==(blocks-1), act_cls=self.act_cls, ndim=self.ndim, ks=self.ks, **kwargs)
              for i in range(blocks)])
        
    def forward(self, x):
        x = self.stem(x)

        c1 = self.layer1(x)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)

        return c4, c1 #, self.head(c4)


def _xresnet(pretrained, expansion, layers, **kwargs):
    # TODO pretrain all sizes. Currently will fail with non-xrn50
    url = 'https://s3.amazonaws.com/fast-ai-modelzoo/xrn50_940.pth'
    if deeplabhead: res = XResNetDeepLabHead(ResBlock, expansion, layers, **kwargs)
    else: res = XResNet(ResBlock, expansion, layers, **kwargs)
    if pretrained: raise ValueError('No pretrained models.')
    return res

def xresnet18 (pretrained=False, **kwargs): return _xresnet(pretrained, 1, [2, 2,  2, 2], **kwargs)
def xresnet50 (pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 4,  6, 3], **kwargs)


def SEBlock(expansion, ni, nf, groups=1, reduction=16, stride=1, **kwargs):
    return ResBlock(expansion, ni, nf, stride=stride, groups=groups, reduction=reduction, nh1=nf*2, nh2=nf*expansion, **kwargs)

def SEResNeXtBlock(expansion, ni, nf, groups=32, reduction=16, stride=1, base_width=4, **kwargs):
    w = math.floor(nf * (base_width / 64)) * groups
    return ResBlock(expansion, ni, nf, stride=stride, groups=groups, reduction=reduction, nh2=w, **kwargs)

def SABlock(expansion, ni, nf, groups=1, sha=True, stride=1, **kwargs):
    return ResBlock(expansion, ni, nf, stride=stride, groups=groups, sha=sha, nh1=nf*2, nh2=nf*expansion, **kwargs)

def SAResNeXtBlock(expansion, ni, nf, groups=32, sha=True, stride=1, base_width=4, **kwargs):
    w = math.floor(nf * (base_width / 64)) * groups
    return ResBlock(expansion, ni, nf, stride=stride, groups=groups, sha=sha, nh2=w, **kwargs)

def ECABlock(expansion, ni, nf, groups=1, eca=True, stride=1, **kwargs):
    return ResBlock(expansion, ni, nf, stride=stride, groups=groups, eca=eca, nh1=nf*2, nh2=nf*expansion, **kwargs)

def ECAResNeXtBlock(expansion, ni, nf, groups=32, eca=True, stride=1, base_width=4, **kwargs):
    w = math.floor(nf * (base_width / 64)) * groups
    return ResBlock(expansion, ni, nf, stride=stride, groups=groups, eca=eca, nh2=w, **kwargs)



def xse_resnet34(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(SEBlock,  1, g1, n_out=n_out, **se_kwargs1, **kwargs)

def xse_resnext34(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(SEResNeXtBlock, 1, g1, n_out=n_out, **se_kwargs2, **kwargs)

def xse_resnet50(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(SEBlock, 4, g1, n_out=n_out, **se_kwargs1, **kwargs)

def xse_resnext50(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet 
    return Model(SEResNeXtBlock, 4, g1, n_out=n_out, **se_kwargs2, **kwargs)

def xsa_resnet50(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(SABlock, 4, g1, n_out=n_out, sha=True, **kwargs)

def xsa_resnext50(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(SAResNeXtBlock, 4, g1, n_out=n_out, groups=32, sha=True, **kwargs)

def xeca_resnet34(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(ECABlock, 1, g1, n_out=n_out, eca=True, **kwargs)

def xeca_resnext34(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(ECAResNeXtBlock, 1, g1, n_out=n_out, groups=32, eca=True, **kwargs)

def xeca_resnet50(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):  
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return XResNet(ECABlock, 4, g1, n_out=n_out, eca=True, **kwargs)

def xeca_resnext50(n_out=1000, pretrained=False, deeplabhead=False, **kwargs):
    Model = XResNetDeepLabHead if deeplabhead else XResNet
    return Model(ECAResNeXtBlock, 4, g1, n_out=n_out, groups=32, eca=True, **kwargs)

## DeeplabV3+
Implementation modified from https://github.com/yassouali/pytorch-segmentation

In [ ]:

''' 
-> The Atrous Spatial Pyramid Pooling
'''

def assp_branch(in_channels, out_channels, kernel_size, dilation, act_cls=defaults.activation):
    padding = 0 if kernel_size == 1 else dilation
    return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_channels),
            defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls())

class ASSP(nn.Module):
    def __init__(self, in_channels, output_stride, p=0.5, act_cls=defaults.activation):
        super(ASSP, self).__init__()

        assert output_stride in [8, 16], 'Only output strides of 8 or 16 are suported'
        if output_stride == 16: dilations = [1, 6, 12, 18]
        elif output_stride == 8: dilations = [1, 12, 24, 36]
        
        self.aspp1 = assp_branch(in_channels, 256, 1, dilation=dilations[0], act_cls=act_cls)
        self.aspp2 = assp_branch(in_channels, 256, 3, dilation=dilations[1], act_cls=act_cls)
        self.aspp3 = assp_branch(in_channels, 256, 3, dilation=dilations[2], act_cls=act_cls)
        self.aspp4 = assp_branch(in_channels, 256, 3, dilation=dilations[3], act_cls=act_cls)

        self.avg_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(in_channels, 256, 1, bias=False),
            nn.BatchNorm2d(256),
            defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls())
        
        self.conv1 = nn.Conv2d(256*5, 256, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(256)
        self.act = defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls()
        self.dropout = nn.Dropout(p)

        init_cnn(self)

    def forward(self, x):
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = F.interpolate(self.avg_pool(x), size=(x.size(2), x.size(3)), mode='bilinear', align_corners=True)

        x = self.conv1(torch.cat((x1, x2, x3, x4, x5), dim=1))
        x = self.bn1(x)
        x = self.dropout(self.act(x))

        return x

''' 
-> Decoder
'''

class Decoder(nn.Module):
    def __init__(self, low_level_channels, num_classes, p=0.1, act_cls=defaults.activation):
        super(Decoder, self).__init__()
        self.conv1 = nn.Conv2d(low_level_channels, 48, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(48)
        self.act = defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls()

        # Table 2, best performance with two 3x3 convs
        self.output = nn.Sequential(
            nn.Conv2d(48+256, 256, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls(),
            nn.Conv2d(256, 256, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(256),
            defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls(),
            nn.Dropout(p),
            nn.Conv2d(256, num_classes, 1, stride=1),
        )
        init_cnn(self)

    def forward(self, x, low_level_features):
        low_level_features = self.conv1(low_level_features)
        low_level_features = self.act(self.bn1(low_level_features))
        H, W = low_level_features.size(2), low_level_features.size(3)

        x = F.interpolate(x, size=(H, W), mode='bilinear', align_corners=True)
        x = self.output(torch.cat((low_level_features, x), dim=1))
        return x

'''
-> Deeplab V3 +
'''

class DeepLab(nn.Module):
    def __init__(self, num_classes, backbone, in_channels=2048, output_stride=16, assp_p=0.5, decode_p=0.1,
                 freeze_bn=False, freeze_backbone=False, act_cls=defaults.activation, **_):
                
        super(DeepLab, self).__init__()
        low_level_channels = int(in_channels/8)

        self.backbone = backbone()
        self.ASSP = ASSP(in_channels=in_channels, output_stride=output_stride, p=assp_p, act_cls=act_cls)
        self.decoder = Decoder(low_level_channels, num_classes, p=decode_p, act_cls=act_cls)

    def forward(self, x):
        H, W = x.size(2), x.size(3)
        x, low_level_features = self.backbone(x)
        x = self.ASSP(x)
        x = self.decoder(x, low_level_features)
        x = F.interpolate(x, size=(H, W), mode='bilinear', align_corners=True)
        return x

# More Augmentations

In [ ]:
@patch
def channel_drop(x:TensorImage, replace, channel, dim):
    return x.index_fill_(dim, channel, replace)

class ChannelDrop(RandTransform):
    order = 65 # After Noise 
    "Drop entire channel by replacing it with random solid value or `replace_val`"
    def __init__(self, p=0.1, replace=None, channels=3, dim=-3):
        super().__init__(p=p)
        self.replace = replace
        self.channels = channels-1
        self.dim = dim

    def before_call(self, b, split_idx):
        super().before_call(b, split_idx)
        self.value = random.uniform(0,1) if self.replace is None else self.replace
        self.channel = random.randint(0, self.channels)

    def encodes(self, x:TensorImage):
        return x.channel_drop(self.value, torch.tensor(self.channel, device=x.device, dtype=torch.long), self.dim)

@patch
def random_noise(x:TensorImage, std):
    return x + torch.normal(mean=0, std=std, size=x.shape, device=x.device, dtype=x.dtype)

class RandomNoise(RandTransform):
    "Add random normal noise."
    order = 60
    def __init__(self, p=0.1, std=0.1):
        super().__init__(p=p)
        self.std = std

    def before_call(self, b, split_idx):
        super().before_call(b, split_idx)
        self.rand_std = random.uniform(0,self.std)

    def encodes(self, x:TensorImage):
        return x.random_noise(self.rand_std)

# Multiloss

In [ ]:
from fastxtend.basics import is_listish

# Internal Cell
def init_loss(l, reduction='none', **kwargs):
    "Initiatize loss class or partial loss function"
    return partialler(l, reduction=reduction) if isinstance(l, types.FunctionType) else l(reduction=reduction, **kwargs)

# Cell
class MultiLoss(Module):
    "Combine multiple `loss_funcs` via `reduction`, with optional weighting. Log loss_funcs to metrics via LossMetricsCallback, optionally using `loss_names`."
    def __init__(self, loss_funcs=[], weights=[], loss_kwargs=[], loss_names=[], reduction='mean', loss_reduction='none'):
        store_attr(but='loss_funcs,weights,loss_kwargs,loss_names')
        if len(weights)==0: self.weights = [1]*len(loss_funcs)
        else: self.weights = weights
        if len(loss_kwargs)==0: loss_kwargs = [{}]*len(loss_funcs)
        self.loss_funcs = [init_loss(l, loss_reduction, **k) for l, k in zip(loss_funcs, loss_kwargs)]
        if len(loss_names)==0: loss_names = [l.__name__ for l in loss_funcs]
        self.loss_names = L(loss_names)
        self._reduction,self._loss = reduction,{}

    def forward(self, pred, targ):
        for i, loss_func in enumerate(self.loss_funcs):
            l = TensorBase(self.weights[i]*loss_func(pred, targ))
            if i == 0: loss = TensorBase(torch.zeros_like(targ)).float()
            loss += l
            self._loss[i] = l
        
        return loss.mean() if self._reduction=='mean' else loss.sum() if self._reduction=='sum' else loss

    @property
    def losses(self): return self._loss

    @property
    def reduction(self): return self._reduction

    @reduction.setter
    def reduction(self, r): self._reduction = r

    @delegates(Module.to)
    def to(self, *args, **kwargs):
        device, dtype, non_blocking, convert_to_format = torch._C._nn._parse_to(*args, **kwargs)
        if is_listish(self.weights) or not isinstance(self.weights, torch.Tensor): self.weights = torch.Tensor(self.weights)
        if self.weights.device != device: self.weights = self.weights.to(device=device)
        super().to(*args, **kwargs)

    def activation(self, pred):
        return getattr(self.loss_funcs[0], 'activation', noop)(pred)

    def decodes(self, pred):
        return getattr(self.loss_funcs[0], 'decodes', noop)(pred)

# Internal Cell
class MultiAvgLoss(AvgLoss):
    "Average the MultiLoss losses taking into account potential different batch sizes"
    def __init__(self, i, name):
        store_attr(but='name')
        self._name = name

    def accumulate(self, learn):
        bs = find_bs(learn.yb)
        self.total += learn.to_detach(learn.loss_func.losses[self.i].mean())*bs
        self.count += bs

    @property
    def name(self): return self._name

# Internal Cell
class MultiAvgSmoothLoss(AvgSmoothLoss):
    "Smooth average of the MultiLoss losses (exponentially weighted with `beta`)"
    def __init__(self, i, name, beta=0.98):
        store_attr(but='name')
        self._name = name

    def accumulate(self, learn):
        self.count += 1
        self.val = torch.lerp(to_detach(learn.loss_func.losses[self.i].mean(), gather=False), self.val, self.beta)

    @property
    def name(self): return self._name

# Cell
class MultiLossCallback(Callback):
    run_valid,order = False,Recorder.order+1
    """A fastai friendly metric implemented as a callback so that we can handle use cases where we don't
    want to count tokens marked to be ignored or else not count batches where there are no targs
    """
    def __init__(self, beta=0.98):
        self.beta, = beta,

    def before_fit(self):
        if not isinstance(self.loss_func, MultiLoss):
            raise ValueError("`LossMetricsCallback` requires loss to be `MultiLoss` class")

        train_mets,valid_mets = L(),L()
        for i in range(len(self.loss_func.loss_funcs)):
            train_mets += MultiAvgSmoothLoss(i, self.loss_func.loss_names[i], self.beta)
            valid_mets += MultiAvgLoss(i, self.loss_func.loss_names[i])
        train_mets.map(Self.reset())

        self.recorder._train_mets = train_mets + self.recorder._train_mets
        self.recorder._valid_mets = valid_mets + self.recorder._valid_mets
        self.recorder._train_smooth_mets = train_mets + self.recorder._train_smooth_mets

        train_idx = self.recorder.metric_names.argfirst(lambda o:o=='train_loss')+1
        valid_idx = self.recorder.metric_names.argfirst(lambda o:o=='valid_loss')+1
        pre_names = self.recorder.metric_names[:train_idx]
        mid_names = self.recorder.metric_names[train_idx:valid_idx]
        post_names = self.recorder.metric_names[valid_idx:]

        train_names = self.loss_func.loss_names.map('train_{}')
        valid_names = self.loss_func.loss_names.map('valid_{}')

        self.recorder.metric_names = pre_names + train_names + mid_names + valid_names + post_names
        self.recorder._train_smooth_names = train_names + self.recorder._train_smooth_names

# Loss Functions

In [ ]:
class DiceLoss(Module):
    "Dice loss for segmentation"
    def __init__(self, axis=1, smooth=1e-6, reduction="mean", square_in_union=False):
        store_attr()

    def forward(self, pred, targ):
        targ = self._one_hot(targ, pred.shape[self.axis])
        pred, targ = TensorBase(pred), TensorBase(targ)
        assert pred.shape == targ.shape, 'input and target dimensions differ, DiceLoss expects non one-hot targs'
        pred = self.activation(pred)
        sum_dims = list(range(2, len(pred.shape)))
        inter = torch.sum(pred*targ, dim=sum_dims)
        union = (torch.sum(pred**2+targ, dim=sum_dims) if self.square_in_union
            else torch.sum(pred+targ, dim=sum_dims))
        dice_score = (2. * inter + self.smooth)/(union + self.smooth)
        loss = (1-dice_score)
        return loss.mean() if self.reduction=='mean' else loss.sum() if self.reduction=='sum' else loss

    @staticmethod
    def _one_hot(x, classes, axis=1):
        "Creates one binay mask per class"
        return torch.stack([torch.where(x==c, 1, 0) for c in range(classes)], axis=axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)
    def decodes(self, x):    return x.argmax(dim=self.axis)

# Datablocks

In [ ]:
cloudn_stats = ([2.74e+03,2.84e+03,2.85e+03,3.66e+03], [2.79e+03,2.9e+03,3.16e+03,2.42e+03])

In [ ]:
def get_multcls_dls(bs, item_tfms=None, batch_tfms=None, valid_fold=4):
    df = pd.read_csv(f'{PATH}cloudn_folds.csv')
    df['valid'] = df.fold == valid_fold
    
    if batch_tfms is None: batch_tfms = []
    batch_tfms += [Normalize.from_stats(*cloudn_stats)]

    block = DataBlock(blocks=(GeoImageBlock(tif=False), GeoMultiClassBlock(tif=False)),
                      get_x=[ColReader('chip_id')],
                      get_y=[ColReader('chip_id')],
                      splitter=ColSplitter('valid'),
                      item_tfms=RandomCrop(256),
                      batch_tfms=batch_tfms,
                      n_inp=1)

    vblock = DataBlock(blocks=(GeoImageCropBlock(tif=False), GeoCropMultiClassBlock(tif=False)),
                       get_x=[ColReader('chip_id')],
                       get_y=[ColReader('chip_id')],
                       splitter=ColSplitter('valid'),
                       item_tfms=item_tfms,
                       batch_tfms=batch_tfms,
                       n_inp=1)
    
    dls = block.dataloaders(df, bs=bs, num_workers=min(8, num_cpus()))
    vdls = vblock.dataloaders(df, bs=bs//4, num_workers=min(8, num_cpus()))

    dls.valid = vdls.valid
    return dls

In [ ]:
def get_segcrops_dls(bs, item_tfms=None, batch_tfms=None, valid_fold=4):
    if FULL: df = pd.read_csv(f'{PATH}cloudn_folds.csv')
    else: df = pd.read_csv(f'{PATH}cloudn_folds_sub.csv')
    df['valid'] = df.fold == valid_fold
    
    if batch_tfms is None: batch_tfms = []
    batch_tfms += [Normalize.from_stats(*cloudn_stats)]

    block = DataBlock(blocks=(GeoImageBlock(tif=False), GeoMaskBlock(tif=False)),
                      get_x=[ColReader('chip_id')],
                      get_y=[ColReader('chip_id')],
                      splitter=ColSplitter('valid'),
                      item_tfms=RandomCrop(256),
                      batch_tfms=batch_tfms,
                      n_inp=1)

    vblock = DataBlock(blocks=(GeoImageCropBlock(tif=False), GeoMaskBlock(tif=False)),
                       get_x=[ColReader('chip_id')],
                       get_y=[ColReader('chip_id')],
                       splitter=ColSplitter('valid'),
                       batch_tfms=Normalize.from_stats(*cloudn_stats),
                       n_inp=1)
    
    dls = block.dataloaders(df, bs=bs, num_workers=min(8, num_cpus()))
    vdls = vblock.dataloaders(df, bs=bs//4, num_workers=min(8, num_cpus()))

    dls.valid = vdls.valid
    return dls

# Multiclass Training

In [ ]:
def multiclass_train(run_name, bs, epochs=10, lr=8e-3, wd=1e-2, pct_start=0.75, item_tfms=None, batch_tfms=None, save=True,
                     act_cls=nn.Mish, drop=0.0, next_lr=None, change_by=None, change_time=1, group='xecaRN50 MultiClass 10e', 
                     arch=xeca_resnet50, stem_pool=stem_pool, block_pool=block_pool, sa=False, seeds=[1618, 314, 2998, 6626, 42]):

    for i in range(len(seeds)):
        valid_fold=i
        print(f'Fold {i}')
        
        with less_random(seeds[i]):
            dls = get_multcls_dls(bs, item_tfms=item_tfms, batch_tfms=batch_tfms, valid_fold=valid_fold)

        run = wandb.init(project="cloudn-submissions", name=f'{run_name} Fold {i}', entity='bwarner', group=group,
                        notes=f'bs={bs}, epochs={epochs}, lr={lr}, wd={wd}, pct_start={pct_start}, valid_fold={valid_fold}')

        cbs=[GradientClip()]
        if save: cbs += [SaveModelCallback(monitor='f1_score', comp=np.greater, fname=f'{run_name} Fold {i}')]

        with less_random(seeds[i]):
            model = arch(c_in=4, n_out=20, act_cls=act_cls, stem_pool=stem_pool, block_pool=block_pool, p=drop, sa=sa)
            learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), opt_func=ranger, 
                            metrics=[F1ScoreMulti(), PrecisionMulti(), RecallMulti()],
                            cbs=cbs+[WandbCallback(log=None, log_preds=False, log_model=True)]).to_fp16()
            
        with less_random(seeds[i]):
            learn.fit_flat_cos(epochs, lr, pct_start=pct_start, wd=wd)

        if save:
            # rename weights so they will match the deeplab expected model names
            state = torch.load(f'models/{run_name} Fold {i}.pth', map_location='cpu')
            for key in list(state.keys()):
                state[key.replace('model', 'model.backbone')] = state.pop(key)
            torch.save(state, f'models/{run_name} Rename Fold {i}.pth', pickle_module=pickle, pickle_protocol=2)

        run.finish()
        learn,dls = None,None
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
bs=64
epochs=15
lr=1e-3

multiclass_train('xSA-ResNeXt50 15e bs64 Multiclass Base', bs=bs, epochs=epochs, lr=lr, arch=xsa_resnext50,
                 group='xSA-RNXt50 Multiclass-to-Seg 15-to-80e', wd=1e-4, save=True,
                 batch_tfms=aug_transforms(flip_vert=True, max_rotate=45, max_zoom=1.2, max_warp=0.1, max_lighting=0,
                 xtra_tfms=[ChannelDrop(channels=4), RandomNoise()]))

Fold 0


epoch,train_loss,valid_loss,f1_score,precision_score,recall_score,time
0,0.336749,0.310142,0.881735,0.915409,0.852252,01:23
1,0.301901,0.288155,0.893410,0.917953,0.870822,01:19
2,0.276528,0.278260,0.898883,0.928527,0.871235,01:18
3,0.265052,0.265648,0.907341,0.904498,0.910742,01:17
4,0.270410,0.283613,0.893663,0.932170,0.858821,01:18
5,0.263718,0.247111,0.914428,0.917598,0.911461,01:19
6,0.244727,0.247019,0.918282,0.896258,0.941719,01:18
7,0.242850,0.232683,0.924152,0.906658,0.942446,01:18
8,0.233029,0.229362,0.923660,0.909836,0.938069,01:18
9,0.234941,0.223137,0.924635,0.930670,0.919001,01:18


Better model found at epoch 0 with f1_score value: 0.881734707768806.
Better model found at epoch 1 with f1_score value: 0.8934095168826328.
Better model found at epoch 2 with f1_score value: 0.8988832918677184.
Better model found at epoch 3 with f1_score value: 0.9073413751307668.
Better model found at epoch 5 with f1_score value: 0.9144275126971253.
Better model found at epoch 6 with f1_score value: 0.9182818167392135.
Better model found at epoch 7 with f1_score value: 0.9241516274507395.
Better model found at epoch 9 with f1_score value: 0.9246349611276325.
Better model found at epoch 11 with f1_score value: 0.9247206987765126.
Better model found at epoch 12 with f1_score value: 0.9299740134882624.
Better model found at epoch 13 with f1_score value: 0.9329688712455301.
Better model found at epoch 14 with f1_score value: 0.9336633903074414.
Fold 1


epoch,train_loss,valid_loss,f1_score,precision_score,recall_score,time
0,0.336339,0.314929,0.896472,0.857745,0.939739,01:17
1,0.300612,0.284494,0.899546,0.902093,0.897577,01:17
2,0.276992,0.255716,0.913594,0.922984,0.904665,01:17
3,0.276346,0.261422,0.909689,0.930618,0.890000,01:18
4,0.262229,0.266458,0.907939,0.915446,0.901022,01:19
5,0.259181,0.263014,0.910476,0.863224,0.963586,01:19
6,0.250738,0.224922,0.927552,0.913725,0.941919,01:19
7,0.254064,0.284875,0.900088,0.838875,0.973493,01:18
8,0.234216,0.214730,0.931164,0.939833,0.922777,01:18
9,0.226353,0.233997,0.924834,0.899297,0.953743,01:17


Better model found at epoch 0 with f1_score value: 0.8964716576403123.
Better model found at epoch 1 with f1_score value: 0.8995461907124467.
Better model found at epoch 2 with f1_score value: 0.9135944894153699.
Better model found at epoch 6 with f1_score value: 0.9275515846900817.
Better model found at epoch 8 with f1_score value: 0.9311640815179677.
Better model found at epoch 12 with f1_score value: 0.9323201338277078.
Better model found at epoch 13 with f1_score value: 0.9357738774084094.
Fold 2


...

Better model found at epoch 0 with f1_score value: 0.8990006498426976.
Better model found at epoch 1 with f1_score value: 0.9027113110738568.
Better model found at epoch 4 with f1_score value: 0.9130994058711007.
Better model found at epoch 7 with f1_score value: 0.9195859247945896.
Better model found at epoch 8 with f1_score value: 0.9260756878240631.
Better model found at epoch 9 with f1_score value: 0.9266499437862675.
Better model found at epoch 12 with f1_score value: 0.9340634861337925.
Fold 3


epoch,train_loss,valid_loss,f1_score,precision_score,recall_score,time
0,0.331932,0.306103,0.892214,0.874556,0.910926,01:17
1,0.297355,0.269806,0.906395,0.887115,0.927296,01:18
2,0.275573,0.268193,0.904621,0.925359,0.885060,01:17
3,0.278884,0.322799,0.875621,0.954202,0.810141,01:20
4,0.263481,0.245887,0.913349,0.932387,0.895745,01:19
5,0.253051,0.228076,0.920799,0.932978,0.909298,01:17
6,0.251165,0.219948,0.927422,0.929948,0.925603,01:17
7,0.250116,0.234285,0.924562,0.904136,0.946205,01:17
8,0.234789,0.220404,0.928298,0.919705,0.938237,01:17
9,0.232512,0.224383,0.925047,0.928036,0.922575,01:17


Better model found at epoch 0 with f1_score value: 0.8922141606849147.
Better model found at epoch 1 with f1_score value: 0.9063945657431993.
Better model found at epoch 4 with f1_score value: 0.9133494728832208.
Better model found at epoch 5 with f1_score value: 0.9207988604642843.
Better model found at epoch 6 with f1_score value: 0.9274215600011386.
Better model found at epoch 8 with f1_score value: 0.9282975635461698.
Better model found at epoch 11 with f1_score value: 0.9305191977031037.
Better model found at epoch 13 with f1_score value: 0.9348138564568039.
Better model found at epoch 14 with f1_score value: 0.9362351900428045.
Fold 4


epoch,train_loss,valid_loss,f1_score,precision_score,recall_score,time
0,0.318884,0.316103,0.891375,0.860283,0.925915,01:17
1,0.282526,0.315173,0.885464,0.927515,0.847496,01:17
2,0.280154,0.270862,0.906346,0.908649,0.904618,01:19
3,0.263402,0.268123,0.910010,0.883595,0.938434,01:18
4,0.256217,0.294182,0.888763,0.925753,0.856356,01:18
5,0.241908,0.309298,0.898440,0.844088,0.961782,01:17
6,0.246645,0.274495,0.904456,0.908908,0.900865,01:17
7,0.239814,0.252842,0.917649,0.935745,0.900438,01:17
8,0.227787,0.300498,0.884354,0.947405,0.829847,01:17
9,0.228341,0.237238,0.921514,0.916479,0.926815,01:17


Better model found at epoch 0 with f1_score value: 0.8913753050184038.
Better model found at epoch 2 with f1_score value: 0.9063455743916133.
Better model found at epoch 3 with f1_score value: 0.9100098848962679.
Better model found at epoch 7 with f1_score value: 0.9176493234056528.
Better model found at epoch 9 with f1_score value: 0.9215137573709884.
Better model found at epoch 11 with f1_score value: 0.9230227640311055.
Better model found at epoch 12 with f1_score value: 0.9275706266581002.
Better model found at epoch 13 with f1_score value: 0.9317050917037368.


# Segmentation Training

In [ ]:
def seg_train(run_name, bs, epochs=80, lr=1e-3, wd=1e-2, pct_start=0.75, item_tfms=None, batch_tfms=None, valid_fold=4, save=False,
              deep_act_cls=nn.Mish, assp_p=0.5, decode_p=0.1, next_lr=None, change_by=None, change_time=1, group='xecaRN50 MultiClass 10e', 
              arch=xeca_resnet50, stem_pool=stem_pool, block_pool=block_pool, sa=False, arch_act_cls=nn.Mish, 
              load_name='xSA-ResNeXt50 15e bs64 Multiclass Base Rename', seeds=[1618, 314, 2998, 6626, 42]):

    for i in range(len(seeds)):
        valid_fold=i
        print(f'Fold {i}')
        
        with less_random(seeds[i]):
            dls = get_segcrops_dls(bs, item_tfms=item_tfms, batch_tfms=batch_tfms, valid_fold=valid_fold)

        run = wandb.init(project="cloudn-submissions", name=f'{run_name} Fold {i}', entity='bwarner', group=group,
                        notes=f'bs={bs}, epochs={epochs}, lr={lr}, wd={wd}, pct_start={pct_start}, valid_fold={valid_fold}')

        loss = MultiLoss([LabelSmoothingCrossEntropy, DiceLoss], loss_kwargs=[{'axis':1},{}], 
                    weights=[1., 1.], loss_names=['ls_cross_entropy', 'dice_loss'], loss_reduction='mean')

        cbs=[MultiLossCallback(), CombineGeoCallback(), GradientClip()]
        if save: cbs += [SaveModelCallback(monitor='jaccard_coeff', comp=np.greater, fname=f'{run_name} Fold {i}')]

        with less_random(seeds[i]):
            backbone = partial(arch, c_in=4, n_out=20, act_cls=arch_act_cls, stem_pool=stem_pool, block_pool=block_pool, sa=sa, deeplabhead=True)
            model = DeepLab(2, backbone, assp_p=assp_p, decode_p=decode_p, act_cls=deep_act_cls)
            learn = Learner(dls, model, loss_func=loss, opt_func=ranger, 
                            metrics=[JaccardCoeff(), Dice()],
                            cbs=cbs+[WandbCallback(log=None, log_preds=True, log_model=True, n_preds=12)]).to_fp16()

        learn.load(f'{load_name} Fold {i}', strict=False)

        with less_random(seeds[i]):
            learn.fit_flat_cos(epochs, lr, pct_start=pct_start, wd=wd)

        run.finish()
        learn,dls = None,None
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
bs=64
epochs=80
lr=1e-3

seg_train('DeepLab xSA-ResNeXt50 80e bs64 Multiclass-to-Seg Base', bs=bs, epochs=epochs, lr=lr, arch=xsa_resnext50, 
          group='xSA-RNXt50 Multiclass-to-Seg 15-to-80e', wd=1e-4, save=True, pct_start=0.5,
          batch_tfms=aug_transforms(flip_vert=True, max_rotate=45, max_zoom=1.2, max_warp=0.1, max_lighting=0,
                                    xtra_tfms=[ChannelDrop(channels=4), RandomNoise()]))

Fold 0


epoch,train_loss,train_ls_cross_entropy,train_dice_loss,valid_loss,valid_ls_cross_entropy,valid_dice_loss,jaccard_coeff,dice,time
0,0.894273,0.440704,0.453569,0.808032,0.393841,0.414191,0.828222,0.906041,02:12
1,0.843211,0.410527,0.432684,0.820440,0.403849,0.416591,0.818156,0.899985,02:11
2,0.809475,0.389652,0.419823,0.764665,0.387566,0.377100,0.837072,0.911311,02:09
3,0.786480,0.376224,0.410256,0.732765,0.370336,0.362429,0.860298,0.924903,02:09
4,0.788923,0.381091,0.407832,0.753549,0.370358,0.383191,0.855621,0.922194,02:09
5,0.787699,0.379211,0.408488,0.786193,0.386135,0.400059,0.837142,0.911353,02:10
6,0.760494,0.360945,0.399549,0.721894,0.357114,0.364780,0.867408,0.928997,02:10
7,0.759062,0.362797,0.396265,0.729572,0.368438,0.361134,0.862062,0.925922,02:09
8,0.754486,0.360574,0.393912,0.740319,0.363900,0.376420,0.865728,0.928033,02:09
9,0.747655,0.355942,0.391713,0.707835,0.356585,0.351250,0.875795,0.933786,02:09


Better model found at epoch 0 with jaccard_coeff value: 0.8282222889657994.
Better model found at epoch 2 with jaccard_coeff value: 0.83707213655886.
Better model found at epoch 3 with jaccard_coeff value: 0.8602975696065156.
Better model found at epoch 6 with jaccard_coeff value: 0.8674078706622291.
Better model found at epoch 9 with jaccard_coeff value: 0.8757951701882729.
Better model found at epoch 13 with jaccard_coeff value: 0.8780829259708557.
Better model found at epoch 14 with jaccard_coeff value: 0.8790056406700795.
Better model found at epoch 18 with jaccard_coeff value: 0.8831592992055081.
Better model found at epoch 19 with jaccard_coeff value: 0.8834701264271617.
Better model found at epoch 26 with jaccard_coeff value: 0.8841827913652879.
Better model found at epoch 28 with jaccard_coeff value: 0.8849020349894925.
Better model found at epoch 30 with jaccard_coeff value: 0.8876527206095074.
Better model found at epoch 37 with jaccard_coeff value: 0.8885291519183003.
Better

epoch,train_loss,train_ls_cross_entropy,train_dice_loss,valid_loss,valid_ls_cross_entropy,valid_dice_loss,jaccard_coeff,dice,time
0,0.883143,0.433834,0.449309,0.799498,0.402191,0.397307,0.832780,0.908761,02:10
1,0.835021,0.406466,0.428555,0.770694,0.387045,0.383650,0.840100,0.913102,02:11
2,0.810684,0.391017,0.419667,0.760693,0.385682,0.375012,0.852254,0.920234,02:10
3,0.787396,0.377064,0.410332,0.769935,0.405512,0.364423,0.834818,0.909974,02:14
4,0.777335,0.373458,0.403877,0.736715,0.365357,0.371358,0.867008,0.928767,02:11
5,0.770257,0.370087,0.400169,0.720155,0.360340,0.359816,0.866415,0.928427,02:12
6,0.767200,0.365237,0.401963,0.730919,0.355294,0.375625,0.866182,0.928293,02:11
7,0.766284,0.366651,0.399633,0.728000,0.353942,0.374057,0.871123,0.931123,02:13
8,0.753006,0.360900,0.392106,0.728245,0.366014,0.362231,0.860531,0.925038,02:11
9,0.744242,0.353276,0.390966,0.700305,0.358032,0.342272,0.879259,0.935751,02:11


Better model found at epoch 0 with jaccard_coeff value: 0.8327797048380502.
Better model found at epoch 1 with jaccard_coeff value: 0.8400995122017069.
Better model found at epoch 2 with jaccard_coeff value: 0.8522539011039554.
Better model found at epoch 4 with jaccard_coeff value: 0.8670075968148014.
Better model found at epoch 7 with jaccard_coeff value: 0.8711231252731187.
Better model found at epoch 9 with jaccard_coeff value: 0.8792586836336503.
Better model found at epoch 12 with jaccard_coeff value: 0.8837851341936729.
Better model found at epoch 15 with jaccard_coeff value: 0.8838740472834229.
Better model found at epoch 16 with jaccard_coeff value: 0.8840735644453379.
Better model found at epoch 22 with jaccard_coeff value: 0.8868805206212182.
Better model found at epoch 24 with jaccard_coeff value: 0.888608729214288.
Better model found at epoch 31 with jaccard_coeff value: 0.8907643283941726.
Better model found at epoch 46 with jaccard_coeff value: 0.8929605315981256.
Better

...

Better model found at epoch 0 with jaccard_coeff value: 0.8083961781139939.
Better model found at epoch 1 with jaccard_coeff value: 0.8388195318435332.
Better model found at epoch 2 with jaccard_coeff value: 0.8517147960486074.
Better model found at epoch 3 with jaccard_coeff value: 0.862816800232352.
Better model found at epoch 4 with jaccard_coeff value: 0.8692650853300034.
Better model found at epoch 7 with jaccard_coeff value: 0.8714037739405738.
Better model found at epoch 12 with jaccard_coeff value: 0.8760318098686927.
Better model found at epoch 15 with jaccard_coeff value: 0.8765502685478693.
Better model found at epoch 21 with jaccard_coeff value: 0.8839532052054068.
Better model found at epoch 28 with jaccard_coeff value: 0.8882379723540135.
Better model found at epoch 37 with jaccard_coeff value: 0.8893826109832446.
Better model found at epoch 40 with jaccard_coeff value: 0.8913514831350379.
Better model found at epoch 43 with jaccard_coeff value: 0.8936338380341151.
Better

epoch,train_loss,train_ls_cross_entropy,train_dice_loss,valid_loss,valid_ls_cross_entropy,valid_dice_loss,jaccard_coeff,dice,time
0,0.899072,0.442295,0.456777,0.815743,0.398656,0.417087,0.822744,0.902753,02:21
1,0.850483,0.411655,0.438827,0.765283,0.382074,0.383208,0.846483,0.916860,02:21
2,0.810104,0.388480,0.421624,0.747886,0.384359,0.363526,0.852337,0.920283,02:21
3,0.798326,0.383152,0.415174,0.735205,0.365466,0.369738,0.864138,0.927118,02:21
4,0.777636,0.372741,0.404895,0.712231,0.367513,0.344718,0.865216,0.927738,02:20
5,0.768105,0.367913,0.400192,0.697823,0.340761,0.357062,0.877013,0.934477,02:21
6,0.771683,0.370969,0.400713,0.696219,0.346859,0.349359,0.877261,0.934618,02:17
7,0.767871,0.365263,0.402609,0.688384,0.345177,0.343207,0.880898,0.936678,02:16
8,0.759407,0.361741,0.397667,0.690505,0.344150,0.346355,0.879365,0.935811,02:19
9,0.756870,0.361906,0.394964,0.688058,0.338281,0.349777,0.882033,0.937319,02:18


Better model found at epoch 0 with jaccard_coeff value: 0.8227441089833312.
Better model found at epoch 1 with jaccard_coeff value: 0.8464834866307889.
Better model found at epoch 2 with jaccard_coeff value: 0.8523365431642373.
Better model found at epoch 3 with jaccard_coeff value: 0.8641379198988882.
Better model found at epoch 4 with jaccard_coeff value: 0.8652161391932162.
Better model found at epoch 5 with jaccard_coeff value: 0.8770127844306845.
Better model found at epoch 6 with jaccard_coeff value: 0.8772606238648007.
Better model found at epoch 7 with jaccard_coeff value: 0.8808982234263589.
Better model found at epoch 9 with jaccard_coeff value: 0.8820331783701998.
Better model found at epoch 12 with jaccard_coeff value: 0.8883293904168177.
Better model found at epoch 21 with jaccard_coeff value: 0.8899241336528324.
Better model found at epoch 25 with jaccard_coeff value: 0.8918456044275582.
Better model found at epoch 31 with jaccard_coeff value: 0.8922930020496306.
Better m

epoch,train_loss,train_ls_cross_entropy,train_dice_loss,valid_loss,valid_ls_cross_entropy,valid_dice_loss,jaccard_coeff,dice,time
0,0.873319,0.427019,0.446300,0.829144,0.410350,0.418794,0.814918,0.898022,02:15
1,0.828578,0.398812,0.429766,0.807554,0.401059,0.406496,0.832752,0.908745,02:14
2,0.816528,0.393018,0.423510,0.802161,0.404521,0.397640,0.843668,0.915206,02:14
3,0.785320,0.374813,0.410507,0.770521,0.393733,0.376788,0.842937,0.914776,02:14
4,0.766293,0.367464,0.398829,0.751863,0.378924,0.372939,0.854400,0.921484,02:14
5,0.755731,0.360378,0.395352,0.731309,0.367937,0.363372,0.864156,0.927129,02:15
6,0.753979,0.360057,0.393921,0.729396,0.362196,0.367200,0.869313,0.930088,02:15
7,0.746052,0.354974,0.391078,0.733625,0.373186,0.360439,0.863749,0.926894,02:15
8,0.746048,0.353985,0.392064,0.721599,0.361404,0.360194,0.872899,0.932137,02:16
9,0.749152,0.356166,0.392987,0.719753,0.354176,0.365577,0.872936,0.932158,02:15


Better model found at epoch 0 with jaccard_coeff value: 0.8149180399769606.
Better model found at epoch 1 with jaccard_coeff value: 0.832752471662957.
Better model found at epoch 2 with jaccard_coeff value: 0.8436675371663992.
Better model found at epoch 4 with jaccard_coeff value: 0.8544000738546504.
Better model found at epoch 5 with jaccard_coeff value: 0.8641562840804397.
Better model found at epoch 6 with jaccard_coeff value: 0.869312777424237.
Better model found at epoch 8 with jaccard_coeff value: 0.8728992300557086.
Better model found at epoch 9 with jaccard_coeff value: 0.8729364929771875.
Better model found at epoch 11 with jaccard_coeff value: 0.8744316189295928.
Better model found at epoch 12 with jaccard_coeff value: 0.8748808521851301.
Better model found at epoch 13 with jaccard_coeff value: 0.8794794547320912.
Better model found at epoch 16 with jaccard_coeff value: 0.8809463947459112.
Better model found at epoch 18 with jaccard_coeff value: 0.8821311189436605.
Better mo